# TJ Graph

## 필요 라이브러리 로드

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import os, math

In [2]:
import platform
from matplotlib import font_manager, rc

# 한글 폰트 지정
path = "C:/Windows/Fonts/malgun.ttf"
if platform.system() == "Windows":
    font_name = font_manager.FontProperties(fname=path).get_name()
    rc('font', family=font_name)
elif platform.system()=="Darwin":
    rc('font', family='AppleGothic')
else:
    print("Unknown System")

In [3]:
import matplotlib
matplotlib.rcParams['axes.unicode_minus'] = False # 그래프에서 음수 표현 가능하도록 설정.

## 파일 불러오기

In [35]:
site_name = 'tj'

In [36]:
# 전 주 파일 날짜
start_date = '2022-04-25'
end_date = '2022-05-01'

In [37]:
# 비교할 파일 날짜(일주일)
new_start_date = '2022-05-02'
new_end_date = '2022-05-08'
new_dates_df = pd.date_range(new_start_date, new_end_date).astype(str)
new_dates_df

Index(['2022-05-02', '2022-05-03', '2022-05-04', '2022-05-05', '2022-05-06',
       '2022-05-07', '2022-05-08'],
      dtype='object')

In [38]:
code_path = 'c:/data/Wantreez/Crawling/music'
code_path

'c:/data/Wantreez/Crawling/music'

In [39]:
# 실시간 순위 파일 폴더로 이동
live_folder = f'/crawled_data/live_{site_name}/'
try:
    os.chdir(code_path + live_folder)
except:
    pass

In [40]:
file_list = os.listdir()
for i, s in enumerate(file_list):
    print(i, s)

0 live_tj_20220425.xlsx
1 live_tj_20220426.xlsx
2 live_tj_20220427.xlsx
3 live_tj_20220428.xlsx
4 live_tj_20220429.xlsx
5 live_tj_20220430.xlsx
6 live_tj_20220501_100024.xlsx
7 live_tj_20220502_100124.xlsx
8 live_tj_20220503_100146.xlsx
9 live_tj_20220504_100159.xlsx
10 live_tj_20220505_100213.xlsx
11 live_tj_20220506_100225.xlsx
12 live_tj_20220507_100236.xlsx
13 live_tj_20220508_100245.xlsx
14 live_tj_20220509_100256.xlsx
15 live_tj_20220510_100307.xlsx
16 live_tj_20220511.xlsx


## 수집한 파일 모두 합치기

In [41]:
total_df = pd.DataFrame()
for one in file_list:
    temp = pd.read_excel(one)
    total_df = pd.concat([total_df, temp], ignore_index=True)
total_df.head()

,날짜,순위,곡,가수
0,2022-04-25,1,취중고백,김민석
1,2022-04-25,2,사랑인가봐(사내맞선OST),멜로망스
2,2022-04-25,3,사랑은늘도망가(신사와아가씨OST),임영웅
3,2022-04-25,4,호랑수월가,탑현
4,2022-04-25,5,너의번호를누르고,#안녕(Prod.영화처럼)


In [42]:
total_df['날짜'] = total_df['날짜'].astype(str)

In [43]:
before_df = total_df[ (total_df['날짜'] >= start_date) & (total_df['날짜'] <= end_date)].reset_index(drop=True)
before_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 700 entries, 0 to 699
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   날짜      700 non-null    object
 1   순위      700 non-null    int64 
 2   곡       700 non-null    object
 3   가수      700 non-null    object
dtypes: int64(1), object(3)
memory usage: 22.0+ KB


In [44]:
new_df = total_df[ (total_df['날짜'] >= new_start_date) & (total_df['날짜'] <= new_end_date)].reset_index(drop=True)
new_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 700 entries, 0 to 699
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   날짜      700 non-null    object
 1   순위      700 non-null    int64 
 2   곡       700 non-null    object
 3   가수      700 non-null    object
dtypes: int64(1), object(3)
memory usage: 22.0+ KB


## 1주일 동안 새로 진입한 곡 추출

In [45]:
title_list = before_df['곡'].unique()

In [46]:
pop_title_list = []
pop_artist_list = []
pop_date_df = pd.DataFrame()
for new_one_date in new_dates_df:
    new_date_df = total_df[total_df['날짜'] == new_one_date].reset_index(drop=True)
    new_title_list = new_date_df['곡'].unique()

    for one_title in new_title_list:
        if one_title not in title_list:
            pop_title_list.append(one_title)
            pop_artist_list.append(new_df[new_df['곡'] == one_title]['가수'].values[0]) # 문제 : 같은 곡명, 다른 가수는 구분이 어려움
    print(new_one_date)
    print( len(pop_title_list), len(pop_artist_list) )

    for two_title, two_artist in zip(pop_title_list, pop_artist_list):
        temp = new_df[ (new_df['곡'] == two_title) & (new_df['가수'] == two_artist)]
        pop_date_df = pd.concat([pop_date_df, temp], ignore_index=True)
        pop_date_df = pop_date_df.drop_duplicates(['날짜', '곡', '가수'], ignore_index=True)

2022-05-02
1 1
2022-05-03
2 2
2022-05-04
5 5
2022-05-05
7 7
2022-05-06
10 10
2022-05-07
13 13
2022-05-08
17 17


In [47]:
pop_date_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17 entries, 0 to 16
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   날짜      17 non-null     object
 1   순위      17 non-null     int64 
 2   곡       17 non-null     object
 3   가수      17 non-null     object
dtypes: int64(1), object(3)
memory usage: 672.0+ bytes


In [49]:
pop_df = pop_date_df.drop_duplicates(['곡', '가수']).reset_index(drop=True)
pop_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   날짜      8 non-null      object
 1   순위      8 non-null      int64 
 2   곡       8 non-null      object
 3   가수      8 non-null      object
dtypes: int64(1), object(3)
memory usage: 384.0+ bytes


## 파일로 출력

In [52]:
pop_df.to_excel('20220509_new_songs.xlsx', index=False, encoding='utf-8')

# 코드 마지막